In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

# Set up plotting preferences
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully")


Libraries loaded successfully


In [3]:
# Load the filtered dataset
df = pd.read_csv('/home/jovyan/work/data/processed/filtered_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"Countries included: {df['Country Name'].unique()}")
print(f"Data period: {df.columns[4:-1].min()} - {df.columns[4:-1].max()}")
print(f"Total indicators: {df['Indicator Name'].nunique()}")


Dataset shape: (4797, 64)
Countries included: ['Bangladesh' 'India' 'Pakistan']
Data period: 1960 - 2018
Total indicators: 1599


In [4]:
# Filter for water/sanitation mortality data
wash_mortality = df[df['Indicator Code'] == 'SH.STA.WASH.P5'].copy()

print(f"Water/sanitation mortality data shape: {wash_mortality.shape}")
print("\n=== Basic Data Information ===")
print(wash_mortality[['Country Name', 'Indicator Name', 'Indicator Code']].head())

# Check if data exists
if wash_mortality.empty:
    print("WARNING: No data found for indicator SH.STA.WASH.P5")
    print("Available indicators containing 'wash' or 'water':")
    water_related = df[df['Indicator Name'].str.contains('water|wash', case=False, na=False)]
    print(water_related[['Indicator Name', 'Indicator Code']].drop_duplicates())
else:
    print(f"\n✓ Successfully found {len(wash_mortality)} rows of water/sanitation mortality data")


Water/sanitation mortality data shape: (3, 64)

=== Basic Data Information ===
     Country Name                                     Indicator Name  \
859    Bangladesh  Mortality rate attributed to unsafe water, uns...   
2458        India  Mortality rate attributed to unsafe water, uns...   
4057     Pakistan  Mortality rate attributed to unsafe water, uns...   

      Indicator Code  
859   SH.STA.WASH.P5  
2458  SH.STA.WASH.P5  
4057  SH.STA.WASH.P5  

✓ Successfully found 3 rows of water/sanitation mortality data


In [5]:
# Transform year columns to rows (melt operation)
year_columns = [col for col in wash_mortality.columns if col.isdigit()]
print(f"Year columns available: {len(year_columns)} years from {min(year_columns)} to {max(year_columns)}")

wash_long = wash_mortality.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Mortality_Rate'
)

# Convert data types
wash_long['Year'] = wash_long['Year'].astype(int)
wash_long['Mortality_Rate'] = pd.to_numeric(wash_long['Mortality_Rate'], errors='coerce')

# Remove missing values
wash_clean = wash_long.dropna(subset=['Mortality_Rate']).copy()

print(f"\nTransformed data shape: {wash_clean.shape}")
print(f"Available year range: {wash_clean['Year'].min()} - {wash_clean['Year'].max()}")
print(f"Non-null data points: {len(wash_clean)}")


Year columns available: 59 years from 1960 to 2018

Transformed data shape: (3, 6)
Available year range: 2016 - 2016
Non-null data points: 3


In [6]:
# Investigate why year range is 2016-2016
print("=== Data Availability Investigation ===")
print(f"Shape of wash_mortality before melt: {wash_mortality.shape}")
print(f"Year columns: {year_columns[:10]}...") # Show first 10 year columns

# Check what data exists in the original wide format
if not wash_mortality.empty:
    print("\n=== Non-null values by year (original data) ===")
    for year in year_columns:
        non_null_count = wash_mortality[year].notna().sum()
        if non_null_count > 0:
            print(f"{year}: {non_null_count} countries have data")
            
    # Show actual values for years with data
    print("\n=== Actual values in years with data ===")
    for year in year_columns:
        if wash_mortality[year].notna().any():
            print(f"\n{year}:")
            for idx, row in wash_mortality.iterrows():
                if pd.notna(row[year]):
                    print(f"  {row['Country Name']}: {row[year]}")

# Basic statistical summary
print("\n=== Overall Statistical Summary ===")
if not wash_clean.empty:
    print(wash_clean['Mortality_Rate'].describe())
    
    print("\n=== Country-wise Statistics ===")
    country_stats = wash_clean.groupby('Country Name')['Mortality_Rate'].agg([
        'count', 'mean', 'std', 'min', 'max'
    ]).round(2)
    country_stats.columns = ['Data Points', 'Mean', 'Std Dev', 'Min', 'Max']
    print(country_stats)
    
    # Data availability analysis
    print("\n=== Data Availability by Country ===")
    for country in wash_clean['Country Name'].unique():
        country_data = wash_clean[wash_clean['Country Name'] == country]
        print(f"{country}: {country_data['Year'].min()} - {country_data['Year'].max()} ({len(country_data)} data points)")
        
    # Missing data analysis
    print(f"\n=== Missing Data Analysis ===")
    total_possible = len(wash_clean['Country Name'].unique()) * len(year_columns)
    actual_data = len(wash_clean)
    missing_percentage = ((total_possible - actual_data) / total_possible) * 100
    print(f"Total possible data points: {total_possible}")
    print(f"Actual data points: {actual_data}")
    print(f"Missing data: {missing_percentage:.1f}%")
else:
    print("No data available for analysis")


=== Overall Statistical Summary ===
count     3.000000
mean     16.700000
std       4.186884
min      11.900000
25%      15.250000
50%      18.600000
75%      19.100000
max      19.600000
Name: Mortality_Rate, dtype: float64

=== Country-wise Statistics ===
              Data Points  Mean  Std Dev   Min   Max
Country Name                                        
Bangladesh              1  11.9      NaN  11.9  11.9
India                   1  18.6      NaN  18.6  18.6
Pakistan                1  19.6      NaN  19.6  19.6

=== Data Availability by Country ===
Bangladesh: 2016 - 2016 (1 data points)
India: 2016 - 2016 (1 data points)
Pakistan: 2016 - 2016 (1 data points)

=== Missing Data Analysis ===
Total possible data points: 177
Actual data points: 3
Missing data: 98.3%


In [ ]:
# Let's explore related indicators if this one has limited data
print("=== Exploring Related Water/Sanitation Indicators ===")

# Search for water, sanitation, or hygiene related indicators
related_indicators = df[df['Indicator Name'].str.contains(
    'water|sanitation|hygiene|wash', case=False, na=False
)]['Indicator Name'].unique()

print(f"Found {len(related_indicators)} related indicators:")
for i, indicator in enumerate(related_indicators, 1):
    print(f"{i}. {indicator}")

# If our main indicator has limited data, let's also look at these alternatives
if len(wash_clean) <= 3:  # Very limited data
    print(f"\n=== Limited data for main indicator (only {len(wash_clean)} data points) ===")
    print("Let's examine alternative indicators with more data availability...")
    
    # Check data availability for each related indicator
    for indicator_name in related_indicators[:5]:  # Check first 5 related indicators
        temp_data = df[df['Indicator Name'] == indicator_name]
        if not temp_data.empty:
            indicator_code = temp_data['Indicator Code'].iloc[0]
            # Quick check of data availability
            temp_long = temp_data.melt(
                id_vars=['Country Name', 'Indicator Code'],
                value_vars=year_columns,
                var_name='Year',
                value_name='Value'
            )
            temp_clean = temp_long.dropna(subset=['Value'])
            
            if len(temp_clean) > 0:
                print(f"\n{indicator_name} ({indicator_code}):")
                print(f"  Data points: {len(temp_clean)}")
                print(f"  Year range: {temp_clean['Year'].astype(int).min()} - {temp_clean['Year'].astype(int).max()}")
                print(f"  Countries: {temp_clean['Country Name'].unique()}")


In [ ]:
# Conditional analysis based on data availability
if len(wash_clean) > 0:
    print("=== Analysis of Available Data ===")
    
    # Simple visualization for the available data
    if len(wash_clean) <= 5:  # Very limited data - simple bar chart
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(wash_clean)), wash_clean['Mortality_Rate'], 
                color=['blue', 'red', 'green'][:len(wash_clean)])
        plt.xticks(range(len(wash_clean)), 
                  [f"{row['Country Name']}\n{row['Year']}" for _, row in wash_clean.iterrows()],
                  rotation=45)
        plt.title('Water/Sanitation Related Mortality Rate\n(per 100,000 population)', 
                 fontsize=14, fontweight='bold')
        plt.ylabel('Mortality Rate (per 100,000)')
        plt.tight_layout()
        plt.show()
        
        # Display the actual values
        print("\n=== All Available Data Points ===")
        for _, row in wash_clean.iterrows():
            print(f"{row['Country Name']} ({row['Year']}): {row['Mortality_Rate']:.2f} deaths per 100,000")
            
    else:  # More data available - time series analysis
        # Time series plot
        plt.figure(figsize=(14, 8))
        for country in wash_clean['Country Name'].unique():
            country_data = wash_clean[wash_clean['Country Name'] == country]
            plt.plot(country_data['Year'], country_data['Mortality_Rate'], 
                     marker='o', linewidth=2, markersize=6, label=country)

        plt.title('Water/Sanitation Related Mortality Trends\n(per 100,000 population)', 
                 fontsize=16, fontweight='bold')
        plt.xlabel('Year', fontsize=12)
        plt.ylabel('Mortality Rate (per 100,000)', fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Country comparison boxplot
        plt.figure(figsize=(12, 8))
        sns.boxplot(data=wash_clean, x='Country Name', y='Mortality_Rate')
        plt.title('Distribution of Water/Sanitation Mortality Rates by Country', 
                 fontsize=16, fontweight='bold')
        plt.xlabel('Country', fontsize=12)
        plt.ylabel('Mortality Rate (per 100,000)', fontsize=12)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

else:
    print("❌ No data available for the specified indicator SH.STA.WASH.P5")
    print("This could be because:")
    print("1. The indicator is not available for these countries")
    print("2. The data is available for different years not in our dataset")
    print("3. The indicator code might be different")


In [ ]:
# Advanced analysis if sufficient data is available
if len(wash_clean) >= 6:  # Need at least 6 data points for meaningful analysis
    print("=== Advanced Statistical Analysis ===")
    
    # Year-over-year change analysis
    def calculate_change_rate(group):
        group = group.sort_values('Year')
        group['Change_Rate'] = group['Mortality_Rate'].pct_change() * 100
        return group

    wash_with_change = wash_clean.groupby('Country Name').apply(calculate_change_rate).reset_index(drop=True)

    # Visualization of change rates
    plt.figure(figsize=(14, 8))
    for country in wash_with_change['Country Name'].unique():
        country_data = wash_with_change[wash_with_change['Country Name'] == country]
        plt.plot(country_data['Year'], country_data['Change_Rate'], 
                 marker='s', linewidth=2, markersize=5, label=country, alpha=0.8)

    plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    plt.title('Year-over-Year Change in Water/Sanitation Mortality Rates', fontsize=16, fontweight='bold')
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Change Rate (%)', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Average annual change rates
    print("=== Average Annual Change Rates ===")
    avg_change = wash_with_change.groupby('Country Name')['Change_Rate'].mean().sort_values()
    for country, rate in avg_change.items():
        direction = "improvement" if rate < 0 else "worsening"
        print(f"{country}: {rate:.2f}% annually ({direction})")
        
    # Linear regression analysis
    print("\n=== Trend Analysis (Linear Regression) ===")
    fig, axes = plt.subplots(1, len(wash_clean['Country Name'].unique()), figsize=(18, 6))
    if len(wash_clean['Country Name'].unique()) == 1:
        axes = [axes]

    for i, country in enumerate(wash_clean['Country Name'].unique()):
        country_data = wash_clean[wash_clean['Country Name'] == country]
        
        if len(country_data) > 1:  # Need at least 2 points for regression
            # Linear regression
            slope, intercept, r_value, p_value, std_err = stats.linregress(country_data['Year'], country_data['Mortality_Rate'])
            
            # Plot
            axes[i].scatter(country_data['Year'], country_data['Mortality_Rate'], alpha=0.7, s=50)
            axes[i].plot(country_data['Year'], slope * country_data['Year'] + intercept, 'r-', linewidth=2)
            axes[i].set_title(f'{country}\\nSlope: {slope:.3f}/year (R²: {r_value**2:.3f})', fontweight='bold')
            axes[i].set_xlabel('Year')
            axes[i].set_ylabel('Mortality Rate')
            axes[i].grid(True, alpha=0.3)
            
            # Statistical significance
            trend = "decreasing" if slope < 0 else "increasing"
            significance = "significant" if p_value < 0.05 else "not significant"
            print(f"{country}: {slope:.3f} deaths/100k per year ({trend}, {significance}, p={p_value:.4f})")

    plt.tight_layout()
    plt.show()

elif len(wash_clean) > 0:
    print("=== Limited Data Analysis ===")
    print("Insufficient data points for advanced time series analysis.")
    print("Consider collecting more years of data or examining related indicators.")

# Summary and conclusions
print("\\n" + "="*60)
print("                    SUMMARY")
print("="*60)

if len(wash_clean) > 0:
    latest_year = wash_clean['Year'].max()
    latest_data = wash_clean[wash_clean['Year'] == latest_year]

    print(f"\\n📊 Latest available data ({latest_year}):")
    for _, row in latest_data.iterrows():
        print(f"   • {row['Country Name']}: {row['Mortality_Rate']:.2f} deaths per 100,000 population")

    if len(wash_clean) > 3:
        print(f"\\n📈 Key findings:")
        highest_current = latest_data.loc[latest_data['Mortality_Rate'].idxmax(), 'Country Name']
        lowest_current = latest_data.loc[latest_data['Mortality_Rate'].idxmin(), 'Country Name']
        print(f"   • Highest current mortality rate: {highest_current}")
        print(f"   • Lowest current mortality rate: {lowest_current}")
        
        # Overall trend
        overall_trend = wash_clean.groupby('Year')['Mortality_Rate'].mean()
        if len(overall_trend) > 1:
            if overall_trend.iloc[-1] < overall_trend.iloc[0]:
                print(f"   • Overall trend: Improving (decreasing mortality rates)")
            else:
                print(f"   • Overall trend: Worsening (increasing mortality rates)")
    
    print(f"\\n📋 Data quality:")
    print(f"   • Total data points: {len(wash_clean)}")
    print(f"   • Year range: {wash_clean['Year'].min()} - {wash_clean['Year'].max()}")
    print(f"   • Countries with data: {', '.join(wash_clean['Country Name'].unique())}")
else:
    print("\\n❌ No data available for analysis")
    print("Recommendation: Check alternative indicators or data sources")

print("\\n" + "="*60)
